# ViCare QnA specialty classification
## Making predictions
- Title: QnAtag_done-model.py
- Description: make predictions after training model
- Author: Dinh Van Quy | <quy.dinh3195@gmail.com>
- Date created: 2017-04-17
- Version: 0.1
- Usage:
- Notes:
- Python_version: 2.7.13

In [1]:
#Set up parameters
vect_obj = 'QnAtag_logit_TfidfVectorizer_1count-words-removal_lbfgs_vect-export.pkl'
model_file = 'QnAtag_logit_TfidfVectorizer_1count-words-removal_lbfgs_model.pkl'

In [2]:
#Import modules
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from pyvi.pyvi import ViTokenizer
import pandas as pd

#Load trained model
logit_model = joblib.load(model_file)

#Define function that process raw Vietnamese medical question into the form that is able to be inputed into model
def run_prediction():
    question = raw_input('Câu hỏi của bạn dành cho bác sĩ: ')
    question = unicode(question, encoding='utf-8')
    
    #Vectorize question
    vect = joblib.load(vect_obj)
    question = pd.Series(question)
    question = vect.transform(question)
    
    #Do the predicting
    predict_spec = logit_model.predict_proba(question)
    proba = predict_spec[0]
    proba = pd.Series(proba)
    
    #Format results
    labels = logit_model.classes_
    labels = pd.Series(labels)
    classes_proba = pd.concat([labels, proba], axis=1, keys=['Chuyên khoa', 'Xác suất'])
    classes_proba_top10 = classes_proba.sort_values(by='Xác suất', ascending=False)[:5]
    return classes_proba_top10

In [23]:
run_prediction()

Câu hỏi của bạn dành cho bác sĩ: Xin hỏi bác sĩ. Tôi có đi khám ở bệnh viện huyện đông anh, có kết luận tôi bị hạch ở cổ Nhưng tôi muốn khám chi tiết hơn, muốn chọc dịch hạch xem có tế bào ác tính hay ko Vậy tôi nên làm như thế nào Xin camt ơn ạ


,Chuyên khoa,Xác suất
63,Ung bướu,0.221217
46,Nội tiết,0.210039
53,Tai - Mũi - Họng,0.150415
17,Khám bệnh,0.061525
52,Sản phụ khoa,0.047381


In [4]:
questions = ['Chào bác sỹ e bi rach ra o mu ban chân bi nhiễm trung roi nam vien dieu tri va da rut chi dc 2 ttuan roi ah vet thuong da lien nhung khi cho chan tiep dat thi nhu máu dc don xuong va da chuyen sang mau tim den nhu vay co sao ko ah',
             'Chào bác sĩ em muốn đặt lịch bác sĩ khám ơ bệnh viện bạch mai thì làm thế nào.',
            'Cho cháu hỏi có thể đặt lịch khám trước được không ạ',
            'Xin hỏi em bị cao huyết áp thì phương pháp chữa là như thế nào ạ',
            'Bac si ơi cho e hỏi.có người anh vừa bị tai nạn giao thông.a bị va chạm ở đầu.bây giờ anh rất chán ăn.e thì lo mà không biết nên mua gì để tẩm bổ. Bác sĩ tư vấn cho ạ',
            'Tôi đang bị rụng tóc theo mảng và đã uống Maxhair, vitamin B5 và Biotin liệu có được không ạ? Cho tôi hỏi thêm là tôi dùng dầu có nhiều dưỡng và tinh dầu có được không ạ? hay gội bồ kết thôi ạ? Tôi xin cám ơn!']

## Conclusion
The cross-validation results above infer that Logistic Regression should be the chosen technique for this problem of auto-tagging specialties at medical questions from ViCare users.

#### Current best record:
>- Accuracy with all training data: <font color='0083e4'>**75.109%**</font>, runtime: <font color='orange'>**126s**</font>
>- Cross-validation score: <font color='0083e4'>**68.943%**</font>, runtime: <font color='orange'>**474s**</font>

Actually the preprocessing data has not been done well enough. Inputing the raw data into model yields an **Accuracy Score** of <font color='0083e4'>**73.800%**</font>, runtime of <font color='orange'>**109s**</font>, with **Cross-validation Score** of <font color='0083e4'>**67.791%**</font>, <font color='orange'>**391s**</font> respectively. This means that current preprocessing data effort only leads to just a <font color='green'>**1.152%**</font> increase in performance.

#### Possible improvements:
- Handle non-accented Vietnamese. Current model performs quite poorly.
- Try more techniques (advanced ones like neural network, ...) or more sophisticated rules for params
- Try harder preprocessing data
    + Create a dictionary for interpreting mistyped or abbreviated words
    + Regroup specialties (so that the number of tags reduces)
    + Translate question into English to make use of English-processing native capabilities
    + Train two different models, one processes Vietnamese, one processes non-accented Vietnamese
- Input the preprocessed data to AWS model to make it learn better then compare again?

#### Notes:
- A really weird thing is that though its cross-validation accuracy remains at ~69%, model using pyvi.pyvi.ViTokenizer to tokenize Vietnamese words produces strange results when using model to predict.
- <font color='red'>Is there possibly any way to get a target variable containing multiple tags, train the model on it and get multiple tags in result?
- If the model is implemented then what is the mechanics that makes it automatically learn new questions (model upgrading)?
- Is there a mechanics to enable human to teach the machine? Kind of correcting the model when it makes prediction mistakes? </font>